Import Statements

In [1]:
from IPython.display import HTML, display
import torch
import multiprocessing

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

Installations

In [3]:
!pip install gradio transformers accelerate bitsandbytes sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

In [5]:
def generate(input_text, minimum_length, maximum_length, beam_search, temperature):
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
  output = model.generate(input_ids, min_length = minimum_length, max_new_tokens = maximum_length, length_penalty = 1.8, num_beams=beam_search, no_repeat_ngram_size = 3, temperature = temperature, top_k = 150, top_p = 0.92, repetition_penalty = 2.4)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [8]:
import gradio as gr

examples = [
    ["Answer the following question by detailing your reasoning: Is Pluto a planet?"],
    ["Q: Can Barack Obama have a conversation with George Washington? Give the rationale before answering."], 
    ["Answer the following question: What is the boiling point of water?"], 
    ["Translate to German: I love eating a good flan!"], 
    ["Generate a cooking recipe to prepare a sachertorte:"], 
    ["The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title 't held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to each a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is n•w taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second t Ilest free-standing structure in France after the Millau Viaduct. Question: Which material is the tower made of?"], 
    ["Answer the following question by reasoning step by step. The world consists of 12 triangles. If we loose 3 triangles, and destroy 4 triangles"]
]

title = "Chat GPT using Flan-T5-Large and Gradio UI"

def inference(text, minimum_length, maximum_length, Frequency_penalty, temperature):
  return generate(text, minimum_length, maximum_length, Frequency_penalty, temperature)

io = gr.Interface(
    fn=inference, 
    inputs=[gr.Textbox(lines=4), gr.Slider(10,500), gr.Slider(20,1000), gr.Slider(0,2, step=2), gr.Slider(0, 1)],
    outputs = [
        gr.Textbox(lines=2, label="output")],title = title, examples = examples, css= """body{background-color: Purple}.input_text input{background-color green !important;}""")
io.launch(share = True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f05f4f5a-fdb1-4dc5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
